```
data_neural = {
    'user1': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1, 2],
        'test': [3, 4]
    },
    'user2': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1],
        'test': [2]
    },
    ...
}
```

In [1]:
import gzip
import pandas as pd
import random

yjmob1 = 'yjmob100k-dataset1.csv.gz'
yjmob_df = pd.read_csv(yjmob1, compression='gzip')
uids = yjmob_df['uid'].unique()

rand_indicies = [random.randint(0, len(uids)) for _ in range(1000)]
selected_uids = [uid for uid in uids[rand_indicies]]
df = yjmob_df[yjmob_df['uid'].isin(selected_uids)] 

# Location Data
def spatial_token(x, y):
    return (x-1)+(y-1)*200

df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)

/var/folders/hx/xp23lpqx4ndfxcvp3qj_bdgr0000gn/T/ipykernel_58353/678693794.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)


In [10]:
# import pandas

# df_train = pd.read_csv('df_train.csv')
# df_test = pd.read_csv('df_test.csv')
# df = pd.concat([df_train, df_test], ignore_index=True)

In [4]:
data_neural = {}
WINDOW_SIZE = 10
STEP_SIZE = 100

index = 0

for uid, group in df.groupby('uid'):
    group = group.head(STEP_SIZE)
    sessions = {}
    for i in range(0, len(group), WINDOW_SIZE):
        # Using min to avoid index out of range for the last window
        sessions[i // WINDOW_SIZE] = list(zip( group['x'][i:i + WINDOW_SIZE], 
                                             group['t'][i:i + WINDOW_SIZE]))
    
    # Define the split indices
    num_sessions = len(sessions)
    train_cutoff = int(num_sessions * 0.8)
    
    train_keys = list(sessions.keys())[:train_cutoff]
    test_keys = list(sessions.keys())[train_cutoff:]

    data_neural[index] = {
        'sessions': sessions,
        'train': train_keys,
        'test': test_keys
    }
    
    index += 1

In [5]:
output = {}

In [6]:
uid_list = [int(i) for i in range(len(data_neural))]

In [7]:
output['data_neural'] = data_neural
output['uid_list'] = uid_list

In [8]:
# Save the dictionary to a pickle file
import pickle

with open('DeepMove-master/data/my_data_neural.pk', 'wb') as f:
    pickle.dump(output, f)

https://github.com/vonfeng/DeepMove?tab=readme-ov-file